In [7]:
from pymongo import MongoClient
import pandas as pd
from bson.objectid import ObjectId
import numpy as np
from collections import defaultdict
from tqdm import tnrange, tqdm_notebook
import csv

In [4]:
db = MongoClient().thesis

In [5]:
for num, doc in enumerate(db.final_db.find()):
    print(str(num) + " " + str(doc["_id"]), end="\r")
    if type(doc["topics"]) != dict:
        topics = {str(k): v for k, v in doc["topics"]}
        db.final_db.update_one({"_id": ObjectId(doc["_id"])}, {'$set': {'topics': topics}})

In [31]:
db.final_db.find_one({"_id": ObjectId("5489df6c37a808115d1ee694")})

{'_id': ObjectId('5489df6c37a808115d1ee694'),
 'title': 'Nutricia открестилась от сотрудницы, попавшейся на взятке в Омске',
 'url': 'http://www.omskinform.ru/news/61941',
 'topics': {'0': 0.00012195121951219497,
  '1': 0.00012195121951219497,
  '2': 0.00012195121951219497,
  '3': 0.00012195121951219497,
  '4': 0.00012195121951219497,
  '5': 0.00012195121951219497,
  '6': 0.00012195121951219497,
  '7': 0.00012195121951219497,
  '8': 0.04307686939911471,
  '9': 0.00012195121951219497,
  '10': 0.5949088533419596,
  '11': 0.00012195121951219497,
  '12': 0.00012195121951219497,
  '13': 0.00012195121951219497,
  '14': 0.00012195121951219497,
  '15': 0.00012195121951219497,
  '16': 0.00012195121951219497,
  '17': 0.00012195121951219497,
  '18': 0.00012195121951219497,
  '19': 0.00012195121951219497,
  '20': 0.00012195121951219497,
  '21': 0.20656089654701712,
  '22': 0.00012195121951219497,
  '23': 0.00012195121951219497,
  '24': 0.00012195121951219497,
  '25': 0.00012195121951219497,
  '26'

In [36]:
db.merged.find_one({"url": "http://www.omskinform.ru/news/66169cecc-25ff-4a20-f507-5b16c9cce528"})

In [38]:
errors = 0
for num, doc in enumerate(db.final_db.find()):
    print(str(num) + " " + str(doc["url"]), end="\r")
    if "content_raw" not in doc:
        try:
            content_raw = db.merged.find_one({"url": doc["url"]})["content"]
        except Exception as err:
            errors += 1
            print("Ошибка")
        db.final_db.update_one({"_id": ObjectId(doc["_id"])}, {'$set': {'content_raw': content_raw}})

Ошибкаhttp://www.omskinform.ru/news/66169cecc-25ff-4a20-f507-5b16c9cce528


In [9]:
from pymongo import MongoClient, DESCENDING, ASCENDING
# from bson.objectid import ObjectId
import csv
from tqdm import tqdm


final_db = MongoClient().thesis.final_db
merged = MongoClient().thesis.merged


def topics_by_id(id):
    TOPICS_NUMBERING = {
        0: "Accidents with children",
        1: "Local authorities: appointments, resignations & statements",
        2: "*Urban development (misc)",
        3: "IT & military high tech",
        4: "Fires",
        5: "Urban demography & housing payments",
        6: "Police actions drug, alcohol & counterfeit money crimes",
        7: "Omsk region industrial developemnt",
        8: "Ads of banking services",
        9: "Holidays and VIP-weddingss",
        10: "Trials on economic crime",
        11: "Public transport and traffic",
        12: "Weather",
        13: "Yury Gamburg resignation",
        14: "*NATO warships in Black sea, Russian rocket launch & contests",
        15: "Regional parliament activities",
        16: "Urban landscaping & greening",
        17: "Schools, orphanages & child charity",
        18: "Car sales",
        19: "Car accidents",
        20: "Education",
        21: "Real estate: contruction",
        22: "Art and literature",
        23: "Street & bridge reconstruction & maintenance",
        24: "Housing & the case of disabled Akhmetov",
        25: "Stray dogs & doghunters",
        26: "Urban events & openings",
        27: "Crimea accession",
        28: "Concerts",
        29: "Sport, hockey",
        30: "Hockey",
        31: "Control & regulation of enterprises",
        32: "Regional taxes & fuel prices",
        33: "Missing persons announcements",
        34: "Theaters & festivals",
        35: "*Omsk media, plants & animals",
        36: "Local authorities: appointments, resignations & statements",
        37: "Macroeconomic events: currency rates & oil prices",
        38: "Housing: heating",
        39: "Russia, Ukraine & US international relations",
        40: "Beauty contests & their winners",
        41: "Arbitration court and the Mostovik case",
        42: "*Military holidays",
        43: "Criminal news",
        44: "Movies and Movie stars",
        45: "Olympic Games 2014 & Omsk athletes",
        46: "Uninterpretable",
        47: "Ivan Klimov’s murder",
        48: "Abridgments of traffic law",
        49: "Regional elections and misc."
    }
    return TOPICS_NUMBERING[id]


def topics_to_dict():
    for doc in tqdm(db.find()):
        # print(type(doc["topics"]) is dict)
        if type(doc["topics"]) is not dict:
            topics_dict = {str(t_num): t_prob for t_num, t_prob in doc["topics"]}
            # print(type(doc["_id"]))
            db.update_one({"_id": doc["_id"]}, {'$set': {"topics": topics_dict}})


def top_docs_for_topic(t_num, limit=50):
    for doc in db.final_db.find().sort("topics.{}".format(t_num), DESCENDING).limit(limit):
        yield doc


def top_docs_comments(t_num, limit=50):
    for doc in db.find({"topics.{}".format(t_num): {"$gte": 0.5}}).sort([
        ("commentsCount", DESCENDING),
        ("topics.{}".format(t_num), DESCENDING)
    ]).limit(int(limit/2)):
        yield doc["topics"][str(t_num)], get_raw_for_docs(doc)

    for doc in db.find({"topics.{}".format(t_num): {"$gte": 0.5}}).sort([
        ("commentsCount", ASCENDING),
        ("topics.{}".format(t_num), DESCENDING)
    ]).limit(int(limit/2)):
        yield doc["topics"][str(t_num)], get_raw_for_docs(doc)

def count_most_probable_text_in_topic(t_num, threshold=0.5):
    return db.find({"topics.{}".format(t_num): {"$gte": threshold}}).count()

# with open("count_most_probable_text_in_topic.csv", "wt") as f:
#     csvwriter = csv.writer(f, delimiter=";")
#     csvwriter.writerow(["Номер темы", "Название темы", "Количество текстов p>=.5"])
#     for t_num in range(50):
#         count = count_most_probable_text_in_topic(t_num)
#         csvwriter.writerow([t_num + 1, topics_by_id(t_num), count])
#         print(t_num, count)



# with open("output_comments.csv", "wt") as f:
#     """
#     Это последний код, который использовался при выгрузке данных
#     В новом файле первые 25 постов в каждой теме отсортированы по количеству комментариев в возрастающем порядке, а последние 25  — в убывающем.
#     Так возможно сравнить наиболее комментируемые и некомментируемые посты в каждой теме.
#     В качестве критерия отнесения поста к теме выбрана вероятность >= 0.5. 
#     Возможна ситуация, когда в теме будет менее 50 постов в случае, если недостаточно постов, превышающих эту границу.
#     """
#     fieldnames = []
#     csvwriter = csv.writer(f, delimiter=";")
#     csvwriter.writerow(["id", "topic_num", "topic_name", "text", "source", "probability", "comments"])

#     for t_num in range(50):
#         print(t_num)
#         t_num += 1
#         csvwriter.writerow([str(t_num) + ".0.0", t_num, topics_by_id(t_num-1), "", "", "", ""])
#         for d_num, doc in enumerate(top_docs_comments(t_num - 1)):
#             d_num += 1
#             text = doc[1]["title"] + " " + doc[1]["content"]
#             text = text.replace(";", ".").replace("\n", " ").replace("\t", " ")
#             csvwriter.writerow(["{}.{}.0".format(t_num, d_num), "",  "", text, doc[1]["source"], doc[0], ""])

#             for c_num, comment in enumerate(doc[1]["comments"]):
#                 c_num += 1
#                 csvwriter.writerow(["{}.{}.{}".format(t_num, d_num, c_num), "", "", "", "", "", comment.replace("\n", " ").replace("\t", " ")])

In [48]:
!pwd

/Users/hun/Google Drive/Projects/Omsk News/workspace/2_new_topics_and_metrics


In [10]:
with open("output.csv", "wt") as f:
    fieldnames = []
    csvwriter = csv.writer(f, delimiter=";")
    csvwriter.writerow(["id", "topic_num", "topic_name", "text", "url", "source", "probability", "comments"])

    for t_num in range(50):
        print(t_num)
        t_num += 1
        csvwriter.writerow([str(t_num) + "_0_0", t_num, topics_by_id(t_num-1), "", "", "", ""])
        for d_num, doc in enumerate(top_docs_for_topic(t_num - 1)):
            d_num += 1
            text = doc["title"] + " " + doc["content_raw"]
            text = text.replace(";", ".").replace("\n", " ").replace("\t", " ").replace("\r", " ")
            try:
                csvwriter.writerow(["{}_{}_0".format(t_num, d_num), "",  "", text, doc["url"], doc["source"], doc["topics"][str(t_num-1)], ""])
            except UnicodeEncodeError as err:
                pass


            for c_num, comment in enumerate(doc["comments"]):
                c_num += 1
                try:
                    csvwriter.writerow(["{}_{}_{}".format(t_num, d_num, c_num), "", "", "", "", "", comment[0].replace("\n", " ").replace("\t", " ").replace("\r", " ")])
                except UnicodeEncodeError as err:
                    pass

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
